In [2]:
!pip install labml-nn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 10.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_

In [3]:
import math
from typing import Dict, Any, Tuple, Optional

import torch
from labml import tracker
from torch import nn

from labml_nn.optimizers import GenericAdaptiveOptimizer, WeightDecay

In [4]:
class Adam(GenericAdaptiveOptimizer):
  def __init__(self, params,
               lr: float = 1e-3, betas: Tuple[float, float] = (0.0, 0.999),
               eps: float = 1e-16,
               weight_decay: WeightDecay = WeightDecay(),
               optimized_update: bool = True,
               defaults: Optional[Dict[str, Any]] = None):
    defaults = {} if defaults is None else defaults
    defaults.update(weight_decay.defaults())
    super().__init__(params, defaults, lr, betas, eps)

    self.weight_decay = weight_decay
    self.optimized_update = optimized_update

'''
`params` is the list of parameters
`lr` is the learning rate α
`betas` is a tuple of (β1​, β2​)
`eps` is ϵ^ or ϵ based on optimized_update
`weight_decay` is an instance of class WeightDecay defined in __init__.py
`optimized_update` is a flag whether to optimize the bias correction of the second moment by doing it after adding ϵ
`defaults` is a dictionary of default for group values. This is useful when you want to extend the class ``Adam`` .
'''

'\n`params` is the list of parameters\n`lr` is the learning rate α\n`betas` is a tuple of (β1\u200b, β2\u200b)\n`eps` is ϵ^ or ϵ based on optimized_update\n`weight_decay` is an instance of class WeightDecay defined in __init__.py\n`optimized_update` is a flag whether to optimize the bias correction of the second moment by doing it after adding ϵ\n`defaults` is a dictionary of default for group values. This is useful when you want to extend the class ``Adam`` .\n'